In [6]:
from pathlib import Path
import pandas as pd

p = Path("/work/data/processed")  # コンテナ内の正しいパス
print(p.exists())                 # Trueになるはず
print(list(p.glob("*"))[:3])      # 最初の3つだけ確認

files = list(p.glob("*.parquet"))  # *.parquet または *.snappy.parquet に注意
print(len(files))

True
[PosixPath('/work/data/processed/part-00000-f07624b2-401d-4bc3-ade0-a8ae982c58b5-c000.snappy.parquet'), PosixPath('/work/data/processed/part-00000-7bf08c82-b8bf-4a93-a28f-4789185354c4-c000.snappy.parquet'), PosixPath('/work/data/processed/part-00001-5d0b89f1-3d52-4ae2-b5da-dd18305c5173-c000.snappy.parquet')]
4


# EDAでデータ概要を把握する

In [9]:
channel_df = pd.read_parquet(files[0])
print(channel_df.shape)
channel_df.head()


(1, 6)


,channel_id,channel_name,published_at,subscriber_count,total_views,video_count
0,UCCPkJMeZHhxKck-EptqQbBA,スキマスイッチ / SUKIMASWITCH,2019-04-04 07:11:52,437000,631905587,647


## channel_dfはチャンネルのメタデータ
- チャンネル名：スキマスイッチ
- 開設日や登録者数、総再生回数及び動画本数を確認

In [22]:
video_df = pd.read_parquet(files[1])
print(video_df.shape)
video_df.head()


(686, 9)


,video_id,title,published_at,view_count,like_count,comment_count,duration,tags,total_seconds
0,--CI9Gndvwc,スキマスイッチ「SUKIMASWITCH 20th Anniversary BEST 『PO...,2023-05-12 03:00:00,17404,493,27,PT2M27S,,147
1,-0TkcSGovqI,#奥田民生 × #スキマスイッチ と #ヴィンテージ #ギター の世界へ！,2022-03-23 03:09:43,6874,269,1,PT47S,,47
2,-29NR48c_Dk,#本日の全力少年＠大分県・別府ビーコンプラザ,2025-02-20 15:51:55,3477,198,5,PT1M10S,,70
3,-7_No7g0oa4,【公式】#86 好評企画！第3回『大橋CASINO』 開催！ピンチをチャンスへ！ スキマスイ...,2022-12-23 10:00:00,27684,890,54,PT13M2S,"スキマスイッチ,SUKIMASWITCH,スキマ,大橋卓弥,大橋,常田真太郎,常田,奏,全力...",782
4,-7nRNRSy0gI,スキマスイッチ「view」副音声付（from スキマスイッチ 2004ファーストツアー “夏...,2022-11-02 09:00:00,22970,754,50,PT4M22S,,262


In [20]:
video_df.columns

Index(['video_id', 'title', 'published_at', 'view_count', 'like_count',
       'comment_count', 'duration', 'tags', 'total_seconds'],
      dtype='object')

## 🎥 動画データ（video_df）概要

| カラム名 | 説明 |
|-----------|------|
| `video_id` | 各動画の一意なID |
| `title` | 動画タイトル |
| `published_at` | 投稿日時 |
| `view_count` | 再生回数 |
| `like_count` | 高評価数 |
| `comment_count` | コメント数 |
| `duration` | 再生時間（ISO形式） |
| `total_seconds` | 再生時間を秒単位に変換した値 |
| `tags` | 動画タグ（リスト形式のことも） |

> YouTubeチャンネル内の各動画の基本的なメタ情報を保持するデータ。

In [23]:
video_df["video_id"].nunique(), len(video_df)

(686, 686)

- 「動画IDの数」と「行数」が完全に一致している → 重複行（同じ video_id を持つレコード）は存在しない
- 総動画本数が647本であるのに対して動画IDが686個あるのはなぜか

In [28]:
video_df[["view_count", "like_count", "comment_count", "total_seconds"]].describe().T

,count,mean,std,min,25%,50%,75%,max
view_count,686.0,719955.424198,8.840995e+06,878.0,9267.75,18076.5,33222.25,202509802.0
like_count,686.0,2981.906706,2.215529e+04,36.0,428.00,602.0,1018.25,446009.0
comment_count,686.0,116.921283,8.592522e+02,0.0,10.00,27.0,60.75,15880.0
total_seconds,686.0,525.249271,5.536432e+02,11.0,52.00,340.5,925.00,6118.0


- 極端に短い（1分未満）の動画もあるが、
- 中央値が5分以上と長く、全体的には通常動画が中心。
- ショートが一部混ざっている可能性はあるが、video_count=647と行数=686の差（+39）はこのあたりの「ショート動画or限定公開動画の扱い」の違いと考えられる。
- ショートやライブが一部含まれている可能性はあるが、全体の傾向を把握するEDAには十分使えるとした。

In [33]:
comment1_df = pd.read_parquet(files[2])
print(comment1_df.shape)
comment1_df.head()


(4201, 6)


,video_id,comment_id,author_display_name,published_at,text_display,like_count
0,16dCkDnhpAA,Ugg-VowAyBdLPXgCoAEC,@増川みずき,2017-06-07 13:42:05,彼女と揉めたらこれきいてすぐ謝ってます<br>よっぽどのことがない限り。<br>ずっと続いて...,214
1,AeMRXJtg500,Ugg-tp_9BX1C8ngCoAEC,@xcogj7898,2016-12-10 08:05:58,軽度の発達障害もち。職場に馴染めず失敗ばっかりしてたけど今ではフォローして支えてくれる人たち...,284
2,BewXWiygfvU,Ugg1f0_KdyFrNngCoAEC,@nippi0088,2016-09-16 04:28:53,夏にドライブしながら聞きたい曲,94
3,9BCqxGHG69U,Ugg54M5_AZVUKngCoAEC,@ダイス-p8q,2017-02-21 14:19:28,耳が幸せーーーーーーー！！！！,34
4,yw8OmnujQdc,UggFD2fcsf4hOXgCoAEC,@不死川実弥-v5w,2017-02-26 14:59:41,再生した瞬間に歌い出すこの感じ。 たまらん♪,9


### 💬 コメントデータ (`comment_df`) のカラム概要

| カラム名 | 内容 | 備考 |
|-----------|-------|------|
| **video_id** | コメントが付いた動画のID | `video_df` と結合可能（外部キー） |
| **comment_id** | 各コメントを一意に識別するID | 重複なし、一意なキー |
| **author_display_name** | コメント投稿者の表示名 | 匿名／ハンドルネームが多い |
| **published_at** | コメントの投稿日 | 時系列分析（投稿の推移など）に利用可能 |
| **text_display** | コメント本文 | 自然言語処理の主対象（感情分析・話題抽出など） |
| **like_count** | コメントへの「高評価（いいね）」数 | コメントの反響度合いを表す指標 |

### コメントデータの概要（comment_df）
- 各動画 (video_id) に紐づくコメント情報を保持
- 投稿者名・コメント内容・投稿日・高評価数などを含む
- コメント本文はtext_displayに格納
- コメントの人気度や視聴者の反応分析に利用可能
- video_idをキーにして動画データ(video_df)と結合可能

In [34]:
comment2_df = pd.read_parquet(files[3])
print(comment2_df.shape)
comment2_df.head()


(4194, 6)


,video_id,comment_id,author_display_name,published_at,text_display,like_count
0,TZAjKYUnW9g,Ugg-3mx2ViehhXgCoAEC,@きょかちん,2014-06-15 14:55:02,自然と涙が出てくる素敵な曲。もう何年も聴き続けているけど飽きない。スキマスイッチの魅力が光る...,43
1,gfiGQhDH4ZI,Ugg1D5j2rOGlSXgCoAEC,@toutou30ni,2015-07-04 14:30:36,涙でてくるわー。<br>昔を思いだしてしまった。<br>スキマスイッチの詞は心に響きすぎる・・・,19
2,Tg2BGVTTQzc,Ugg2qIhuznXsKXgCoAEC,@peace7967,2016-06-11 04:30:30,懐かしいみにいったなぁ,31
3,9BCqxGHG69U,Ugg3SUi_8rHDgHgCoAEC,@黒田友梨恵-n9i,2016-07-29 15:54:40,幸せそう。みんな音楽を楽しんで演奏してる。わくわくしてる。人の表情って不思議ですよね。言葉な...,80
4,Tg2BGVTTQzc,Ugg7qkO2q4oeq3gCoAEC,@kumorizorando,2015-07-16 18:06:45,ライブのゴールデンタイムラバーはまた違ってよかったなぁ～,57


### コメントデータに関してはcomment1及びcomment2ともに同じ列構成のため縦結合する

In [41]:
comment_df = pd.concat([comment1_df, comment2_df], ignore_index=True)
print(comment_df.shape)
print(len(comment1_df)+len(comment2_df)==len(comment_df))
comment_df.head(n=10)

(8395, 6)
True


,video_id,comment_id,author_display_name,published_at,text_display,like_count
0,16dCkDnhpAA,Ugg-VowAyBdLPXgCoAEC,@増川みずき,2017-06-07 13:42:05,彼女と揉めたらこれきいてすぐ謝ってます<br>よっぽどのことがない限り。<br>ずっと続いて...,214
1,AeMRXJtg500,Ugg-tp_9BX1C8ngCoAEC,@xcogj7898,2016-12-10 08:05:58,軽度の発達障害もち。職場に馴染めず失敗ばっかりしてたけど今ではフォローして支えてくれる人たち...,284
2,BewXWiygfvU,Ugg1f0_KdyFrNngCoAEC,@nippi0088,2016-09-16 04:28:53,夏にドライブしながら聞きたい曲,94
3,9BCqxGHG69U,Ugg54M5_AZVUKngCoAEC,@ダイス-p8q,2017-02-21 14:19:28,耳が幸せーーーーーーー！！！！,34
4,yw8OmnujQdc,UggFD2fcsf4hOXgCoAEC,@不死川実弥-v5w,2017-02-26 14:59:41,再生した瞬間に歌い出すこの感じ。 たまらん♪,9
5,Tg2BGVTTQzc,UggFXEJRhkdaMXgCoAEC,@小林陽子-n5j,2016-04-07 06:45:14,やっぱスキマは安定してるわ………,148
6,0vHyhZyBtsE,UggKRm0r5vvssHgCoAEC,@kazukicarandmotorcyclefact4042,2014-06-12 14:35:28,8年経っても、全く色褪せない名曲<br>この一言に尽きる。,127
7,TZAjKYUnW9g,UggLKYJts1_rgngCoAEC,@ぶらうぐらな-y2m,2014-01-31 17:55:31,せつない…。<br>こんなにも切ない裏声を響かせる曲なら、心に届かないわけないです。,24
8,bzFYUVCE8Go,UggLZeYhFjTvjXgCoAEC,@spaspabeef,2016-09-20 09:50:43,君の話、螺旋、アーセンの憂鬱のこの3部作は初期の最高傑作,233
9,bzFYUVCE8Go,UggRdPeW7DpkdHgCoAEC,@como5mo,2017-02-20 13:19:17,最後にviewのレモン爆弾が登場してる！,34


In [45]:
video_comment_df = video_df.merge(comment_df, on="video_id", how="left")
print(video_comment_df.shape)
video_comment_df.head()


(8981, 14)


,video_id,title,published_at_x,view_count,like_count_x,comment_count,duration,tags,total_seconds,comment_id,author_display_name,published_at_y,text_display,like_count_y
0,--CI9Gndvwc,スキマスイッチ「SUKIMASWITCH 20th Anniversary BEST 『PO...,2023-05-12 03:00:00,17404,493,27,PT2M27S,,147,NaN,NaN,NaT,NaN,NaN
1,-0TkcSGovqI,#奥田民生 × #スキマスイッチ と #ヴィンテージ #ギター の世界へ！,2022-03-23 03:09:43,6874,269,1,PT47S,,47,NaN,NaN,NaT,NaN,NaN
2,-29NR48c_Dk,#本日の全力少年＠大分県・別府ビーコンプラザ,2025-02-20 15:51:55,3477,198,5,PT1M10S,,70,NaN,NaN,NaT,NaN,NaN
3,-7_No7g0oa4,【公式】#86 好評企画！第3回『大橋CASINO』 開催！ピンチをチャンスへ！ スキマスイ...,2022-12-23 10:00:00,27684,890,54,PT13M2S,"スキマスイッチ,SUKIMASWITCH,スキマ,大橋卓弥,大橋,常田真太郎,常田,奏,全力...",782,NaN,NaN,NaT,NaN,NaN
4,-7nRNRSy0gI,スキマスイッチ「view」副音声付（from スキマスイッチ 2004ファーストツアー “夏...,2022-11-02 09:00:00,22970,754,50,PT4M22S,,262,NaN,NaN,NaT,NaN,NaN


In [46]:
video_comment_df["text_display"].isnull().sum()

np.int64(586)

In [47]:
video_comment_df.isna().mean().sort_values(ascending=False)

comment_id             0.065249
author_display_name    0.065249
published_at_y         0.065249
text_display           0.065249
like_count_y           0.065249
video_id               0.000000
title                  0.000000
published_at_x         0.000000
view_count             0.000000
like_count_x           0.000000
comment_count          0.000000
duration               0.000000
tags                   0.000000
total_seconds          0.000000
dtype: float64